kernal: scanpy

# Set up

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.backends.backend_pdf as mpdf
from matplotlib.pyplot import rc_context

import scanpy as sc
import muon as mu

import warnings
from numba.core.errors import NumbaDeprecationWarning
warnings.filterwarnings(action='once')
warnings.simplefilter(action='once')
warnings.simplefilter(action="ignore", category=NumbaDeprecationWarning)
warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.simplefilter(action="ignore", category=DeprecationWarning)

In [ ]:
sc.settings.verbosity = 0  # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.settings.set_figure_params(dpi=100, frameon=False, figsize=(8, 7), facecolor="white")
sc.logging.print_versions()

In [ ]:
blood_markers_dict = {
    "HSC": ["CD34", "SPINK2", "MLLT3", "HLF", "MECOM", "CDK6", "SELL",
            "CD52", "PROM1", "RUNX1", "HOXA9", "MEIS1", "MYB", "ITGA6"],
    "GP": ["MPO", "AZU1", "SPI1", "LYZ"],
    "MEMP": ["GATA1", "GATA2", "TESPA1", "KLF1", "CTNNB1"],
    "Ery": ["TFRC", "AHSP", "ALAS2", "HBA1", "HBB", "GYPA", "BPGM"],
    "MK": ["ITGA2B", "GP9", "PLEK", 'MPL', 'PECAM1', 'CXCR4', "PPBP", "PF4"],
    "Mast": ["HDC", "CPA3", "LMO4", "CD63", 'ENPP3', "TPSAB1", "TPSB2"],
    "Mono": ["CD14", "FCGR3A", "S100A9", "CD68", "MNDA", "FCN1"],
    "Kupffer": ["CD163", "MS4A7", "C1QA", "MRC1", "CTSB", "MARCO", "CD5L", "VCAM1"],
    "cDC1": ["CLEC9A", "THBD", "XCR1", "BATF3"],
    "cDC2": ["CD1C", "CLEC4A", "CLEC10A"],
    "cDC3": ["FLT3", "VCAN"],
    "pDC": ["JCHAIN", "IRF8", "CLEC4C", "IL3RA", "MPEG1"],
    "B": ["CD79A", "CD79B", "IL7R", "PAX5", "MME", "IGLL1", "IGHM", "IGHD", "CD19", "MS4A1", "IRF4", "DNTT", "RAG1", "RAG2", "CD24", "CD38"],
    "B1": ["CD5", "CD27", "SPN", "CCR10"],
    "NK": ["IL2RG", "NKG7", "PRF1", "GZMA", "KLRB1", "TRBC1", "IL2RB"],
    "ILC": ["RORC", "AHR", "ID2"],
    "T": ["CD2", "CD7", "CD3D", "CD3E", "CD3G", "TRAC", "FOXP3", "TIGIT", "CD4", "CD8A", "CD8B"],
    "Mix": ["KIT", "GATA3", "IL1A", "IL1B", "PTPRC"],
    "Hepa": ["ALB", "AFP"],
    "Endo": ["CDH5", "KDR"],
    "LSEC": ["STAB1", "STAB2", "LYVE1"],
    "Stellate": ["DCN", "COL1A1", "COL3A1", "RBP1"],
    "Epi": ["KRT19"],
    "Cycling": ["MKI67", "TOP2A"]
}

blood_markers_lst = [
  "CD34", "SPINK2", "MLLT3", "HLF", "MECOM", "CDK6", "SELL", "CD52", "PROM1",
  "RUNX1", "HOXA9", "MEIS1", "MYB", "ITGA6", # HSC/MPP
  "MPO", "AZU1", "SPI1", "LYZ", # Granulocyte
  "GATA1", "GATA2", "TESPA1", "KLF1", "CTNNB1", # MEMPs (megakaryocyte-erythroid-mast cell progenitor)
  "TFRC", "AHSP", "ALAS2", "HBA1", "HBB", "GYPA", "BPGM", # Erythroid
  "ITGA2B", "GP9", "PLEK", 'MPL', 'PECAM1', 'CXCR4', "PPBP", "PF4", # Megakaryocytes
  "HDC", "CPA3", "LMO4", "CD63", 'ENPP3', "TPSAB1", "TPSB2", # Mast cells
  "CD14", "FCGR3A", "S100A9", "CD68", "MNDA", "FCN1", # Monocytes
  "CD163", "MS4A7", "C1QA", "MRC1", "CTSB", "MARCO", "CD5L", "VCAM1", # Kupffer cells
  "CLEC9A", "THBD", "XCR1", "BATF3", # cDC1
  "CD1C", "CLEC4A", "CLEC10A", # cDC2
  "FLT3", "VCAN", # cDC3
  "JCHAIN", "IRF8", "CLEC4C", "IL3RA", "MPEG1", # pDCs
  "CD79A", "CD79B", "IL7R", "PAX5", "MME", "IGLL1", "IGHM", "IGHD",
  "CD19", "MS4A1", "IRF4", "DNTT", "RAG1", "RAG2", "CD24", "CD38", # B cells
  "CD5", "CD27", "SPN", "CCR10", # B1
  "IL2RG", "NKG7", "PRF1", "GZMA", "KLRB1", "TRBC1", "IL2RB", # NK
  "RORC", "AHR", "ID2", # ILC3
  "CD2", "CD7", "CD3D", "CD3E", "CD3G", "TRAC", "FOXP3", "TIGIT", "CD4", "CD8A", "CD8B", # T
  "KIT", "GATA3", "IL1A", "IL1B",
  "PTPRC", # CD45
  "ALB", "AFP", # Hepatocytes
  "CDH5", "KDR", # endothelial cells
  "STAB1", "STAB2", "LYVE1", "DCN", # LSECs
  "COL1A1", "COL3A1", "RBP1", # stellate cells
  "KRT19",
  'MKI67', "TOP2A" # cycling
]

suo_science_b = {
  "lymphoid prog.": ["CD34", "SPINK2", "IL7R", "KIT"], # lymphoid prog.
  "PreProB": ["FLT3", "CD19", "VPREB1"], # pre pro B
  "ProB": ["MME", "CDC45", "DHFR", "MKI67"], # pro B
  "LateProB": ["CD27", "RAG1", "DNTT", "VPREB3"], # late pro B
  "Pro -> Pre": ["CD24", "TNFRSF17"], # pro -> pre
  "Pre -> Immature": ["MME", "IDH2", "SPIB", "IL4R", "IGHM"], # pre -> immature
  "Mature": ["IGHD", "MS4A1", "CD40", "FCER2"],  # mature
  "B1": ["CD27", "CD5", "SPN", "CCR10"], # B1
  "PlasmaB": ["JCHAIN", "SDC1", "CD38"],
  "Cycling": ["MKI67", "TOP2A"]
}

suo_science_megak_ery = {
  "Prog.": ["CD34", "SPINK2", "MLLT3"],
  "MK/Ery precur.": ["TESPA1", "GATA2", "FCER1A"], # MegaK/Ery precursors
  "Earyly Ery": ["KLF1", "APOE", "FAM178B"], # early ery
  "Mid Ery": ["BLVRB", "CD36", "OAT"], # mid ery
  "Late Ery": ["GYPA", "GYPB", "SLC4A1"], # late ery
  "YS Ery": ["HBZ", "HBE1"],
  "Early MK": ["HBD", "PF4"], # early megaK
  "Late MK": ["ITGA2B", "ITGB3", "CLK1"], # late megaK
  "Mf_Ery": ["C1QA", "CD163"],
  "Cycling": ["MKI67", "TOP2A"]
}

suo_science_mye = {
  "Prog.": ["CD34", "SPINK2", "MLLT3"],
  "Mye prog.": ["PRSS57", "PRTN3", "AZU1"], # mye prog.
  "Neutrophil": ["ELANE", "DEFA4", "LCN2", "LTF", "ORM1"], # neutrophil
  "Mono": ["CD52", "S100A8", "MS4A6A", "CD14", "CXCR4", "CCR2", "IL1B", "CD300E"], # mono
  "DC prog.": ["ACY3", "TIFAB", "KIF17"], # DC prog.
  "pDC": ["CLEC4C", "JCHAIN", "IRF7"], # pDC
  "ASDC": ["SIGLEC6", "AXL"], # ASDC
  "DC2": ["CLEC10A", "CD1C"], # DC2
  "DC1": ["CLEC9A", "BATF3"], # DC1
  "migratory DC": ["CCR7", "LAMP3"], # migratory DC
  "Langerhans": ["IDO1", "CD207", "CD1A"], # Langerhans cells
  "EO_BASO_Mast": ["CLC", "KIT", "TPSAB1"], # EO_BASO_Mast
  "Mac_LYVE1_high": ["F13A1", "LYVE1", "SPP1"], # mac LYVE1 high
  "Mac iron recycling": ["CD5L", "APOE", "VCAM1"], # mac iron recycling
  "Mac_MHCII_high": ["HLA-DRA", "HLA-DPA1", "CLEC7A"], # mac MHCII high
  "Mac_kupffer_like": ["ENG", "KDR", "CAV1"], # mac kupffer like
  "Mac_TREM2": ["TREM2", "P2RY12"], # mac trem2
  "Mac_TLF": ["TIMD4", "FOLR2"], # mac TLF+
  "Cycling": ["MKI67", "TOP2A"]
}

suo_science_t_ilc = {
    "DN(early)": ["IGLL1", "ST18"],
    "DN(P)": ["TRGC2", "TRDC", "HIVEP3"],
    "DN(Q)": ["RAG1", "RAG2", "TP53INP1", "PTCRA", "RGPD3"],
    "DP(P)": ["SMPD3"],
    "DP(Q)": ["AQP3", "RORC"],
    "abT(entry)": ["CCR9", "SATB1", "TOX2"],
    "Mature_T": ["CCR7", "CD5", "CD27", "CD8A", "CD8B", "CD4"],
    "Treg": ["FOXP3", "CTLA4"],
    "CD8aa": ["PDCD1", "GNG4", "ZNF683"],
    "innate": ["KLRB1", "ZBTB16"],
    "Type3_innate_T": ["CD40LG", "RORC", "ANXA1"],
    "ILC3": ["RORC", "IL1R1", "IL23R", "KIT"],
    "ILC2": ["PTGDR2", "HPGDS", "GATA3"],
    "Type1_innate_T": ["NKG7", "EOMES", "TBX21", "IFNG-AS1"],
    "NK": ["KLRD1", "KLRF1", "NKG7", "NCAM1"],
    "Cycling": ["MKI67", "TOP2A"]
}

# Load data

In [ ]:
work_dir = '/work/DevM_analysis/01.annotation/08.integration_joint'
dataset = "FL_wnn"
new_file, old_file = "v00", "v00"
new_anno, old_anno = "leiden_wnn_3", "leiden_wnn_3"

Load data

In [ ]:
mdata = mu.read(
    f"data/{dataset}_clustered.{old_file}.h5mu"
)
mdata

Modify obs (latest meta info)

In [ ]:
# Load the CSV file
lib_info = pd.read_csv("/work/DevM_analysis/utils/sam_info.DevM.22.10.24.csv")

# Filter the dataframe for rows where 'Tissue' is 'FL' and 'Sex' is not 'Unknown'
lib_info_filtered = lib_info[(lib_info['Tissue'] == 'FL') & (lib_info['Sex'] != 'Unknown')]

# Select specific columns and drop duplicates
df_pcw_batch = lib_info_filtered[['libraryID', 'PCW', 'Batch']].drop_duplicates()
df_pcw_batch['PCW'] = df_pcw_batch.replace({np.nan: 'Missing'})['PCW'].astype(str)
pcw_categories = [str(i) for i in range(5, 19)] + ['Missing']
df_pcw_batch['PCW'] = pd.Categorical(df_pcw_batch['PCW'], categories=pcw_categories, ordered=True)

# mdata
mdata.obs['libraryID'] = mdata['rna'].obs['libraryID'].copy()
mdata.obs['donorID'] = mdata['rna'].obs['donorID'].copy()
mdata.obs['sampleID'] = mdata['rna'].obs['sampleID'].copy()
original_index = mdata.obs.index.copy()
mdata.obs = mdata.obs.merge(df_pcw_batch, on='libraryID', how='left')
mdata.obs.index = original_index
mdata.obs['libraryID'] = pd.Categorical(mdata.obs['libraryID'], categories = df_pcw_batch['libraryID'])
mdata.obs['donorID'] = pd.Categorical(mdata.obs['donorID'], categories = lib_info_filtered['donorID'].unique())
mdata.obs['sampleID'] = pd.Categorical(mdata.obs['sampleID'], categories = lib_info_filtered['sampleID'])

# rna
rna = mdata['rna']
rna.obs['libraryID'] = mdata.obs['libraryID'].copy()
rna.obs['donorID'] = mdata.obs['donorID'].copy()
rna.obs['sampleID'] = mdata.obs['sampleID'].copy()
rna.obs['PCW'] = mdata.obs['PCW'].copy()
rna.obs['Batch'] = mdata.obs['Batch'].copy()

# atac
atac = mdata['atac']
atac.obs['libraryID'] = mdata.obs['libraryID'].copy()
atac.obs['donorID'] = mdata.obs['donorID'].copy()
atac.obs['sampleID'] = mdata.obs['sampleID'].copy()
atac.obs['PCW'] = mdata.obs['PCW'].copy()
atac.obs['Batch'] = mdata.obs['Batch'].copy()

In [ ]:
mdata

# Dotplot

Add new anno

In [ ]:
mdata['rna'].obs[new_anno] = mdata.obs[new_anno]
mdata['atac'].obs[new_anno] = mdata.obs[new_anno]

## RNA

In [ ]:
sc.pl.dotplot(mdata['rna'], var_names=blood_markers_dict, groupby=[new_anno], standard_scale="var")

In [ ]:
sc.pl.dotplot(mdata['rna'], var_names=blood_markers_dict, groupby=[new_anno], standard_scale="var", show=False,
              figsize=(45, len(mdata.obs[new_anno].cat.categories) * 0.35))
plt.savefig(f"{work_dir}/plots/{dataset}_dotplot_rna.{new_file}.pdf", bbox_inches="tight")
plt.close()

suo_science_megak_ery

In [ ]:
sc.pl.dotplot(mdata['rna'], var_names=suo_science_megak_ery, groupby=[new_anno], standard_scale="var")

suo_science_b

In [ ]:
sc.pl.dotplot(mdata['rna'], var_names=suo_science_b, groupby=[new_anno], standard_scale="var")

suo_science_mye

In [ ]:
sc.pl.dotplot(mdata['rna'], var_names=suo_science_mye, groupby=[new_anno], standard_scale="var")

suo_science_t_ilc

In [ ]:
sc.pl.dotplot(mdata['rna'], var_names=suo_science_t_ilc, groupby=[new_anno], standard_scale="var")

# UMAP

Random cells

In [ ]:
np.random.seed(0)
random_indices = np.random.permutation(list(range(mdata.shape[0])))

## Batch mixing

Library/Donor

In [ ]:
with plt.rc_context({"figure.figsize": (12, 9)}):  # Use this to set figure params like size and dpi
    mu.pl.embedding(mdata[random_indices, :], basis='umap', color=["libraryID"], size=1.5, show=False)
    plt.savefig(f"{work_dir}/plots/{dataset}_umap_library.{new_file}.pdf", bbox_inches="tight")

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(30, 8), gridspec_kw={"wspace": 0})
datasets = [mdata, mdata['rna'], mdata['atac']]
titles = ['WNN', "RNA", "ATAC"]
for i, (adata, ax) in enumerate(zip(datasets, axes)):
    if i == 2:
        mu.pl.embedding(adata, basis='umap', color=["libraryID"], size=1.5, ax=ax, show=False, frameon=True, hspace=0)
    else:
        mu.pl.embedding(adata, basis='umap', color=["libraryID"], size=1.5, ax=ax, show=False, frameon=True, hspace=0, legend_loc=None)
    # Add a title for each subplot
    ax.set_title(titles[i])
plt.tight_layout()

plt.savefig(f"{work_dir}/plots/{dataset}_umap_library_perModality.{new_file}.pdf", bbox_inches="tight")

In [ ]:
with plt.rc_context({"figure.figsize": (12, 9)}):  # Use this to set figure params like size and dpi
    mu.pl.embedding(mdata[random_indices, :], basis='umap', color=["donorID"], size=1.5, show=False)
    plt.savefig(f"{work_dir}/plots/{dataset}_umap_donor.{new_file}.pdf", bbox_inches="tight")

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(30, 8), gridspec_kw={"wspace": 0})
datasets = [mdata, mdata['rna'], mdata['atac']]
titles = ['WNN', "RNA", "ATAC"]
for i, (adata, ax) in enumerate(zip(datasets, axes)):
    if i == 2:
        mu.pl.embedding(adata, basis='umap', color=["donorID"], size=1.5, ax=ax, show=False, frameon=True, hspace=0)
    else:
        mu.pl.embedding(adata, basis='umap', color=["donorID"], size=1.5, ax=ax, show=False, frameon=True, hspace=0, legend_loc=None)
    # Add a title for each subplot
    ax.set_title(titles[i])
plt.tight_layout()

plt.savefig(f"{work_dir}/plots/{dataset}_umap_donor_perModality.{new_file}.pdf", bbox_inches="tight")

In [ ]:
with plt.rc_context({"figure.figsize": (12, 9)}):  # Use this to set figure params like size and dpi
    mu.pl.embedding(mdata[random_indices, :], basis='umap', color=["sampleID"], size=1.5, show=False)
    plt.savefig(f"{work_dir}/plots/{dataset}_umap_sample.{new_file}.pdf", bbox_inches="tight")

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(30, 8), gridspec_kw={"wspace": 0})
datasets = [mdata, mdata['rna'], mdata['atac']]
titles = ['WNN', "RNA", "ATAC"]
for i, (adata, ax) in enumerate(zip(datasets, axes)):
    if i == 2:
        mu.pl.embedding(adata, basis='umap', color=["sampleID"], size=1.5, ax=ax, show=False, frameon=True, hspace=0)
    else:
        mu.pl.embedding(adata, basis='umap', color=["sampleID"], size=1.5, ax=ax, show=False, frameon=True, hspace=0, legend_loc=None)
    # Add a title for each subplot
    ax.set_title(titles[i])
plt.tight_layout()

plt.savefig(f"{work_dir}/plots/{dataset}_umap_sample_perModality.{new_file}.pdf", bbox_inches="tight")

batch

In [ ]:
with plt.rc_context({"figure.figsize": (12, 9)}):  # Use this to set figure params like size and dpi
    mu.pl.embedding(mdata[random_indices, :], basis='umap', color=["Batch"], size=1.5, show=False)
    plt.savefig(f"{work_dir}/plots/{dataset}_umap_batch.{new_file}.pdf", bbox_inches="tight")

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(30, 8), gridspec_kw={"wspace": 0})
datasets = [mdata, mdata['rna'], mdata['atac']]
titles = ['WNN', "RNA", "ATAC"]
for i, (adata, ax) in enumerate(zip(datasets, axes)):
    if i == 2:
        mu.pl.embedding(adata, basis='umap', color=["Batch"], size=1.5, ax=ax, show=False, frameon=True, hspace=0)
    else:
        mu.pl.embedding(adata, basis='umap', color=["Batch"], size=1.5, ax=ax, show=False, frameon=True, hspace=0, legend_loc=None)
    # Add a title for each subplot
    ax.set_title(titles[i])
plt.tight_layout()

plt.savefig(f"{work_dir}/plots/{dataset}_umap_batch_perModality.{new_file}.pdf", bbox_inches="tight")

PCW

In [ ]:
with plt.rc_context({"figure.figsize": (12, 9)}):  # Use this to set figure params like size and dpi
    mu.pl.embedding(mdata[random_indices, :], basis='umap', color=["PCW"], size=1.5, show=False)
    plt.savefig(f"{work_dir}/plots/{dataset}_umap_PCW.{new_file}.pdf", bbox_inches="tight")

In [ ]:
mdata.obs["PCW"] = mdata.obs["PCW"].astype('category')
n_ctypes, ncols = len(mdata.obs["PCW"].unique()), 5
nrows = int(np.ceil(n_ctypes / ncols))
sns.set_theme(style="white", font_scale=1.5)
_, axes = plt.subplots(nrows, ncols, figsize=(ncols * 5.5, nrows * 5))
for i, clust in enumerate(mdata.obs["PCW"].cat.categories):
    i += 1
    mu.pl.embedding(mdata[random_indices, :], basis='umap', color=["PCW"], groups=[clust],
               show=False, use_raw=False, ax=axes[int((i - 1) / ncols), (i - 1) % ncols],
               title=f"{clust}", palette=["red"], ncols=ncols, legend_loc=None, size=2)
plt.tight_layout()

Cell cycle

In [ ]:
with plt.rc_context({"figure.figsize": (12, 10)}):  # Use this to set figure params like size and dpi
    mu.pl.embedding(mdata[random_indices, :], basis='umap', color=["rna:Phase", "rna:G2M.Score"], size=2, show=False)
    plt.savefig(f"{work_dir}/plots/{dataset}_umap_cellCycle.{new_file}.pdf", bbox_inches="tight")

## Cluster

Clustering of Diff. resolution

In [ ]:
mu.pl.embedding(mdata[random_indices, :], basis='umap', color=['leiden_wnn_0.5', 'leiden_wnn_1', 'leiden_wnn_2',
                                                               'leiden_wnn_3', 'leiden_wnn_4', 'leiden_wnn_5', 'leiden_wnn_6'],
                size=1, ncols=4, legend_loc="on data")

Leiden res. 3

In [ ]:
with rc_context({"figure.figsize": (11, 10)}):
    mu.pl.embedding(mdata[random_indices, :], basis='umap', color=[new_anno],
                size=1, legend_loc="on data", show=False)
    plt.savefig(f"{work_dir}/plots/{dataset}_umap_cluster.{new_file}.pdf", bbox_inches="tight")

Each cluster

In [ ]:
n_ctypes, ncols = len(mdata.obs[new_anno].unique()), 5
nrows = int(np.ceil(n_ctypes / ncols))
sns.set_theme(style="white", font_scale=1.5)
_, axes = plt.subplots(nrows, ncols, figsize=(ncols * 5.5, nrows * 5))
for i, clust in enumerate(mdata.obs[new_anno].cat.categories):
    i += 1
    mu.pl.embedding(mdata[random_indices, :], basis='umap', color=[new_anno], groups=[clust],
               show=False, use_raw=False, ax=axes[int((i - 1) / ncols), (i - 1) % ncols],
               title=f"{clust}", palette=["red"], ncols=ncols, legend_loc=None, size=2)
plt.tight_layout()

In [ ]:
n_ctypes, ncols = len(mdata.obs[new_anno].unique()), 5
nrows = int(np.ceil(n_ctypes / ncols))
sns.set_theme(style="white", font_scale=1.5)
_, axes = plt.subplots(nrows, ncols, figsize=(ncols * 5.5, nrows * 5))
for i, clust in enumerate(mdata.obs[new_anno].cat.categories):
    i += 1
    mu.pl.embedding(mdata[random_indices, :], basis='umap', color=[new_anno], groups=[clust],
               show=False, use_raw=False, ax=axes[int((i - 1) / ncols), (i - 1) % ncols],
               title=f"{clust}", palette=["red"], ncols=ncols, legend_loc=None, size=2)
plt.tight_layout()
plt.savefig(f"{work_dir}/plots/{dataset}_uamp_clusterSep.{new_file}.png", dpi=300,
            bbox_inches="tight", facecolor="white")
plt.close()

# Dendrogram

In [ ]:
sc.tl.dendrogram(mdata['rna'], groupby=[new_anno], n_pcs=mdata['rna'].obsm["X_harmony"].shape[1], use_rep="X_harmony",
                 cor_method="pearson", linkage_method="complete", optimal_ordering=True)

In [ ]:
with plt.rc_context({"figure.figsize": (25, 5)}):
    sc.pl.dendrogram(mdata['rna'], groupby = new_anno, show=False)
    plt.savefig(f"{work_dir}/plots/{dataset}_dendrogramOfRNA.{new_file}.pdf", bbox_inches="tight")

# Composition

In [ ]:
def compo_plot(data=None, groupby=None, condition=None):
    #groupby_key = "leiden_wnn_0.9"
    #condition_key = rna_anno
    df = pd.crosstab(data[groupby], data[condition])
    df = df.div(df.sum(axis=1), axis=0) * 100.0
    ax = df.plot(
            kind = "bar",
            stacked = True,
            legend = False
        )
    ax.set_xlabel(groupby)
    ax.set_ylabel("Percentage")
    ax.legend(loc="center left", bbox_to_anchor=(1.05, 0.5), ncol=3)
    if len(max(df.index.astype(str), key=len)) >= 5:
        ax.set_xticklabels(ax.get_xticklabels(), rotation=-45, ha='left')

In [ ]:
groupby_key = "leiden_wnn_1"
condition_key = "libraryID"
compo_plot(data=mdata.obs, groupby=groupby_key, condition=condition_key)

In [ ]:
groupby_key = "leiden_wnn_1"
condition_key = "donorID"
compo_plot(data=mdata.obs, groupby=groupby_key, condition=condition_key)

In [ ]:
groupby_key = "leiden_wnn_1"
condition_key = "Batch"
compo_plot(data=mdata.obs, groupby=groupby_key, condition=condition_key)

# QC

In [ ]:
d4p = mdata.obs.copy()

## Cluster size, gene/umi/peak count

In [ ]:
df = d4p.groupby(new_anno).agg({"rna:nFeature_RNA": 'median', 'rna:nCount_RNA': 'median', 'atac:nCount_peaks': 'median', 'atac:nFeature_peaks': 'median'})
df['count'] = d4p[new_anno].value_counts()
df.to_csv(f"{work_dir}/data/{dataset}_clusterSizes_medianCounts.{new_file}.csv")

## Doublet

In [ ]:
my_order = d4p.groupby(by=[new_anno])["rna:scDblFinder.score"].median().sort_values().index
plt.figure(figsize=(len(mdata.obs[new_anno].cat.categories) * 0.3, 5))
p = sns.boxplot(data = d4p, y = 'rna:scDblFinder.score', x = new_anno, order=my_order)
p.set_xticklabels(p.get_xticklabels(), rotation=45, horizontalalignment='center')
plt.show()

In [ ]:
sns.set(rc={"figure.figsize": (9, 30)})
sns.set_theme(style="white", font_scale=1)
sns.boxenplot(d4p, x="rna:scDblFinder.score", y=new_anno,
              order=my_order)
plt.savefig(f"{work_dir}/plots/{dataset}_doubletScore.{new_file}.pdf", bbox_inches="tight")
plt.close()

## RNA

nCount_RNA

In [ ]:
my_order = d4p.groupby(by=[new_anno])["rna:nCount_RNA"].median().sort_values().index
plt.figure(figsize=(len(mdata.obs[new_anno].cat.categories) * 0.3, 5))
p = sns.boxplot(data = d4p, y = "rna:nCount_RNA", x = new_anno, order=my_order,)
plt.yscale('log')
p.set_xticklabels(p.get_xticklabels(), rotation=45, horizontalalignment='right')
plt.show()

nFeature_RNA

In [ ]:
my_order = d4p.groupby(by=[new_anno])["rna:nFeature_RNA"].median().sort_values().index
plt.figure(figsize=(len(mdata.obs[new_anno].cat.categories) * 0.3, 5))
p = sns.boxplot(data = d4p, y = "rna:nFeature_RNA", x = new_anno, order=my_order, )
plt.yscale('log')
p.set_xticklabels(p.get_xticklabels(), rotation=45, horizontalalignment='center')

# Add horizontal lines
plt.axhline(y=300, color='red', linestyle='--', linewidth=2)  # Add horizontal line at y=1000
plt.axhline(y=500, color='red', linestyle='-.', linewidth=2)  # Add horizontal line at y=10000

plt.show()

percent.mt

In [ ]:
my_order = d4p.groupby(by=[new_anno])["rna:percent.mt"].median().sort_values().index
plt.figure(figsize=(len(mdata.obs[new_anno].cat.categories) * 0.3, 5))
p = sns.boxplot(data = d4p, y = "rna:percent.mt", x = new_anno, order=my_order, )
p.set_xticklabels(p.get_xticklabels(), rotation=45, horizontalalignment='center')
plt.show()

percent.rb

In [ ]:
my_order = d4p.groupby(by=[new_anno])["rna:percent.rb"].median().sort_values().index
plt.figure(figsize=(len(mdata.obs[new_anno].cat.categories) * 0.3, 5))
p = sns.boxplot(data = d4p, y = "rna:percent.rb", x = new_anno, order=my_order, )
p.set_xticklabels(p.get_xticklabels(), rotation=45, horizontalalignment='center')
plt.show()

## ATAC

nCount_peaks

In [ ]:
my_order = d4p.groupby(by=[new_anno])["atac:nCount_peaks"].median().sort_values().index
plt.figure(figsize=(len(mdata.obs[new_anno].cat.categories) * 0.3, 5))
p = sns.boxplot(data = d4p, y = 'atac:nCount_peaks', x = new_anno, order=my_order, )
plt.yscale('log')
p.set_xticklabels(p.get_xticklabels(), rotation=45, horizontalalignment='center')
plt.show()

nFeature_peaks

In [ ]:
my_order = d4p.groupby(by=[new_anno])["atac:nFeature_peaks"].median().sort_values().index
plt.figure(figsize=(len(mdata.obs[new_anno].cat.categories) * 0.3, 5))
p = sns.boxplot(data = d4p, y = 'atac:nFeature_peaks', x = new_anno, order=my_order, )
plt.yscale('log')

p.set_xticklabels(p.get_xticklabels(), rotation=45, horizontalalignment='center')
plt.show()

nucleosome_signal

In [ ]:
my_order = d4p.groupby(by=[new_anno])["atac:nucleosome_signal"].median().sort_values().index
plt.figure(figsize=(len(mdata.obs[new_anno].cat.categories) * 0.3, 5))
p = sns.boxplot(data = d4p, y = 'atac:nucleosome_signal', x = new_anno, order=my_order, )
p.set_xticklabels(p.get_xticklabels(), rotation=45, horizontalalignment='center')
plt.show()

TSS.enrichment

In [ ]:
my_order = d4p.groupby(by=[new_anno])["atac:TSS.enrichment"].median().sort_values().index
plt.figure(figsize=(len(mdata.obs[new_anno].cat.categories) * 0.3, 5))
p = sns.boxplot(data = d4p, y = 'atac:TSS.enrichment', x = new_anno, order=my_order, )
p.set_xticklabels(p.get_xticklabels(), rotation=45, horizontalalignment='center')
plt.show()

# Ridge plots

## RNA

In [ ]:
pdf = mpdf.PdfPages(f"{work_dir}/plots/{dataset}_ridgePlots_rna.{new_file}.pdf")
for gene in blood_markers_lst:
    print(gene)
    sns.set_theme(style="white", font_scale=1)
    _, axes = plt.subplots(len(mdata.obs[new_anno].cat.categories), 1,
                           figsize=(12, len(mdata.obs[new_anno].cat.categories) * 1), sharex=True)
    for i, clust in enumerate(mdata.obs[new_anno].cat.categories):
        sns.kdeplot(data=pd.DataFrame(mdata['rna'][mdata.obs[new_anno] == clust, gene].X.todense(),
                                      columns=[gene]), x=gene, ax=axes[i], fill=True, warn_singular=False)
        axes[i].text(0, 0.2, clust, fontweight="bold", ha="left", va="center", transform=axes[i].transAxes)
        axes[i].spines["top"].set_visible(False)
        axes[i].spines["right"].set_visible(False)
        axes[i].spines["left"].set_visible(False)
        axes[i].set_xlabel(None)
        axes[i].set_ylabel(gene)
    plt.tight_layout()
    plt.suptitle(gene, y=1)
    pdf.savefig()
    plt.close()
pdf.close()

# Save

In [ ]:
mdata